In [2]:
import re
import sqlite3

In [3]:
base_name = 'tonmon.sqlite'
table_names = ['ver_B', 'ver_C'] #таблицы, из которых берутся данные для авторазметки
table_name = 'ver_B' #таблица, которую мы размечаем
con = sqlite3.connect(base_name)
cur = con.cursor()

In [ ]:
def get_voc(table_names):
    cur.execute('SELECT DISTINCT SUF, GLOSS, WClass FROM {} GROUP BY SUF, WClass'.format(*table_names))
    voc = cur.fetchall()
    con.commit()
    v_dic = {}
    for suf, gloss, wc in voc:
        if suf and suf != '' and gloss and wc:
            sufs = re.split('=|\+', suf)
            glosses = re.split('\+|=', gloss)
            for s, gl in zip(sufs, glosses):
                if wc == 'BV' or 'NEGPrep' in wc or 'COP' in wc:
                    wc = 'P'
                if wc == 'BN':
                    wc = 'N'
                if (s, wc) not in v_dic:
                    v_dic[(s, wc)] = [gl]
                elif gl not in v_dic[(s, wc)]:
                    v_dic[(s, wc)].append(gl)
                if gl == 'NMNZ' or wc == 'BN':
                    wc = 'N'
                if gl == 'COP':
                    wc = 'P'
                    
    return v_dic

In [ ]:
def auto_gloss(table_names, table_name):
    voc = get_voc(table_names)
    cur.execute('SELECT ID, SUF, WClass FROM {}'.format(table_name))
    vals = cur.fetchall()
    glosses = []
    for i, suf, wc in vals:
        if suf and wc and suf != '':
            sufs = re.split('=|\+', suf)
            if wc == 'BN':
                wc = 'N'
            if wc == 'BV' or 'NEGPrep' in wc or 'COP' in wc:
                wc = 'P'
            gl = []
            for s in sufs:
                if (s, wc) in voc:
                    gl.append('|'.join(voc[(s, wc)]))
                    if voc[(s, wc)] == ['NMNZ']:
                        wc = 'N'
                else:
                    gl.append('?')
            gl = '+'.join(gl)
            cur.execute('UPDATE {} SET AUTO_GLOSS = ? WHERE ID = {}'.format(table_name, i), (gl,))
            con.commit()